THINGS TO DO (IN ORDER)
- import everything
- get the data
- analyzze data
  - oversample if needed
  - clean up anything weird (there shouldn't be much)
- split the data (train and test)
- training data
  - find best parameters for this / different models
- test the data
- get all results
- map results
- train CNN
- train Transformer


In [35]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
import random
import torch
from torch import nn, optim
import math
from IPython import display
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
import pdb
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn import metrics
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torch.utils.data import Dataset
from PIL import Image, ImageEnhance

mps_device = torch.device("mps")


In [36]:
#traindf = pd.read_csv("_classes.csv")
#traindf.describe()
#traindf[""]

In [37]:
# Data loader function to load data into batches

class Data(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        csv_file: Path to the csv file.
        root_dir: Directory with all the images.
        transform: transform to be applied on a sample. optional
        """
        self.frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.frame.iloc[idx, 0])
        image = Image.open(img_name)
        labels = self.frame.iloc[idx, 1:].to_numpy()
        labels = torch.from_numpy(labels.astype('float')).float()

        if self.transform:
            image = self.transform(image)

        return image, labels

transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
])

# Dataset and DataLoader
dataset = Data(csv_file='Acne.v21i.multiclass/test/_classes.csv', root_dir='Acne.v21i.multiclass/test/', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

num_classes = 6
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc1 = nn.Linear(640000, 1000)
        self.fc2 = nn.Linear(1000, num_classes)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

net = Net()
net.to(mps_device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(mps_device)
        labels = labels.to(mps_device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Done Training')


RuntimeError: MPS backend out of memory (MPS allocated: 6.07 GB, other allocations: 2.64 GB, max allowed: 9.07 GB). Tried to allocate 663.06 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
for a_type in _ :
  count = a_type[a_type]
  if count < 300: 
    count_to_change = (0.18 * count) / 3
  else: 
    count_to_change = (0.05 * count) / 3
  change_im_df = traindf[a_type].sample(frac=count_to_change)
  for image in change_im_df :
    sheared_im = image.clone().shear('Black', 20, 30) # should print out and check if this is too awkward..
    pixeliated_im = image.clone().resize(300, 226) # should print out and check if this is too grainy..
    darker_im = ImageEnhance.Brightness(image.clone()).enhance() # should print out and check if this is too dark..
    altered_df = pd.DataFrame({a_type:[sheared_im, pixeliated_im, darker_im]})
    traindf.append(altered_df, ignore_index=True) # might be more complicated than this ... 
